# Hyperparameter tuning for Keras Model with both Dense and Conv layer

### Load libray

In [ ]:
!pip install scikit-optimize
#imports we know we'll need
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.keras.datasets import cifar10
from keras.utils import np_utils
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

import pandas as pd
import random
import numpy as np

     |████████████████████████████████| 100 kB 8.0 MB/s 


### Load data

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 6s 0us/step


Scale the data to between 0 & 1

In [ ]:
X_train = X_train/ 255
X_test = X_test/ 255
print(X_train.min(),X_train.max())

0.0 1.0


Convert the y's to used with softmax function

In [ ]:
y_train.shape

(50000, 1)

In [ ]:
input_shape= X_train[0].shape
print(input_shape)

(32, 32, 3)


In [ ]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

## Checking result against a baseline


In [ ]:
model =Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(16, activation='relu',name = 'input_layer'))
model.add(Dense(16, activation='relu', name="hidden_layer"))

model.add(Dense(10,activation='softmax',name="output_layer"))
model.compile(optimizer = SGD(learning_rate = 0.001), loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 input_layer (Dense)         (None, 16)                131088    
                                                                 
 hidden_layer (Dense)        (None, 16)                272       
                                                        

In [ ]:
early_S = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
blackbox = model.fit(X_train, y_train, batch_size=128, epochs = 30, validation_split=.15, callbacks = [early_S])

Epoch 1/30
333/333 [==============================] - 15s 9ms/step - loss: 2.2473 - accuracy: 0.1476 - val_loss: 2.2160 - val_accuracy: 0.1620
Epoch 2/30
333/333 [==============================] - 2s 7ms/step - loss: 2.1813 - accuracy: 0.1753 - val_loss: 2.2665 - val_accuracy: 0.1420
Epoch 3/30
333/333 [==============================] - 3s 8ms/step - loss: 2.1319 - accuracy: 0.1967 - val_loss: 2.1591 - val_accuracy: 0.1864
Epoch 4/30
333/333 [==============================] - 2s 7ms/step - loss: 2.0918 - accuracy: 0.2112 - val_loss: 2.0947 - val_accuracy: 0.2168
Epoch 5/30
333/333 [==============================] - 2s 7ms/step - loss: 2.0544 - accuracy: 0.2296 - val_loss: 2.0716 - val_accuracy: 0.2463
Epoch 6/30
333/333 [==============================] - 3s 8ms/step - loss: 2.0174 - accuracy: 0.2541 - val_loss: 2.0971 - val_accuracy: 0.2272
Epoch 7/30
333/333 [==============================] - 3s 8ms/step - loss: 1.9827 - accuracy: 0.2806 - val_loss: 1.9839 - val_accuracy: 0.2848
Epoch

In [ ]:
accuracy = model.evaluate(X_test,y_test)[1]
print(accuracy)

313/313 [==============================] - 1s 3ms/step - loss: 1.8045 - accuracy: 0.3565
0.3564999997615814


### Hyperparameter space

In [ ]:
dim_learning_rate = Real(low=1e-4, high=1e-1, prior='log-uniform',
                         name='learning_rate')
dim_num_conv_layers = Integer(low=1, high=3, name='num_conv_layers')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_dense_nodes = Integer(low=1, high=28, name='num_dense_nodes')
dim_activation = Categorical(categories=['relu', 'sigmoid'],
                             name='activation')
dim_batch_size = Integer(low=1, high=128, name='batch_size')

dimensions = [dim_learning_rate,
              dim_num_conv_layers,
              dim_num_dense_layers,
              dim_num_dense_nodes,
              dim_activation,
              dim_batch_size
             ]
default_parameters = [1e-3,1, 1, 13, 'relu',64]

### Create Model

In [ ]:
def create_model(learning_rate, num_conv_layers, num_dense_layers,
                 num_dense_nodes, activation):
    #start the model making process and create our first layer
    model = Sequential()
    model.add(Input(shape=(32,32,3)))
    #create a loop making a new dense layer for the amount passed to this model.
    #naming the layers helps avoid tensorflow error deep in the stack trace.

    for i in range(num_conv_layers):
        name = 'layer_conv_1_{0}'.format(i+1)
        if i == 0: node = 32
        elif i == 1: node = 64
        else: node = 128
        model.add(Conv2D(node, (3, 3), activation='relu', 
                         kernel_initializer='he_uniform', padding='same',
                         name=name))
        name = 'layer_conv_2_{0}'.format(i+1)
        model.add(Conv2D(node, (3, 3), activation='relu', 
                         kernel_initializer='he_uniform', padding='same',
                         name=name))
        model.add(MaxPooling2D((2, 2)))
        
    model.add(Flatten())

    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes,
                 activation=activation,
                        name=name
                 ))
    #add our classification layer.
    model.add(Dense(10,activation='softmax'))
    
    #setup our optimizer and compile
    sgd = SGD(learning_rate=learning_rate)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

### Objective function

In [ ]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_conv_layers, num_dense_layers,  
            num_dense_nodes,activation, batch_size):

    model = create_model(learning_rate=learning_rate,
                         num_conv_layers = num_conv_layers,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         activation=activation
                        )
    early_S = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
    #named blackbox becuase it represents the structure
    blackbox = model.fit(x=X_train,
                        y=y_train,
                        epochs=25,
                        batch_size=batch_size,
                        validation_split=0.15,
                        callbacks = [early_S]
                        )
    #return the validation accuracy for the last epoch.
    accuracy = blackbox.history['val_accuracy'][-1]

    # Print the classification accuracy.
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    ops.reset_default_graph()
    
    return -accuracy

### Run the optimizer

In [ ]:
gp_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=50,
                            noise= 0.01,
                            n_jobs=-1,
                            verbose=True,
                            kappa = 5,
                            x0=default_parameters)

Iteration No: 1 started. Evaluating function at provided point.
Epoch 1/25
665/665 [==============================] - 4s 6ms/step - loss: 2.2175 - accuracy: 0.1679 - val_loss: 2.1806 - val_accuracy: 0.1976
Epoch 2/25
665/665 [==============================] - 3s 5ms/step - loss: 2.1135 - accuracy: 0.2275 - val_loss: 2.0751 - val_accuracy: 0.2365
Epoch 3/25
665/665 [==============================] - 3s 5ms/step - loss: 2.0105 - accuracy: 0.2828 - val_loss: 2.0312 - val_accuracy: 0.2729
Epoch 4/25
665/665 [==============================] - 3s 5ms/step - loss: 1.9014 - accuracy: 0.3293 - val_loss: 1.9398 - val_accuracy: 0.2904
Epoch 5/25
665/665 [==============================] - 3s 5ms/step - loss: 1.8278 - accuracy: 0.3553 - val_loss: 2.0583 - val_accuracy: 0.2635
Epoch 6/25
665/665 [==============================] - 3s 5ms/step - loss: 1.7774 - accuracy: 0.3740 - val_loss: 1.7776 - val_accuracy: 0.3708
Epoch 7/25
665/665 [==============================] - 3s 5ms/step - loss: 1.7357 - a

### The best results

In [ ]:
gp_result.x

[0.04130837784848988, 3, 4, 26, 'relu', 33]

In [ ]:
print("best accuracy was " + str(round(gp_result.fun *-100,2))+"%.")

best accuracy was 75.75%.


### Model results with best tuned parameters

In [ ]:
gp_model = create_model(gp_result.x[0],gp_result.x[1],gp_result.x[2],gp_result.x[3],gp_result.x[4])
early_S = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights= True)
#named blackbox becuase it represents the structure
blackbox = gp_model.fit(x=X_train,
                    y=y_train,
                    epochs=25,
                    batch_size=gp_result.x[5],
                    validation_split=0.15
                    callbacks = [early_S]
                    )
#return the validation accuracy for the last epoch.
accuracy = blackbox.history['val_accuracy'][-1]

Epoch 1/25
1288/1288 [==============================] - 8s 6ms/step - loss: 1.9063 - accuracy: 0.2749 - val_loss: 1.5787 - val_accuracy: 0.4164
Epoch 2/25
1288/1288 [==============================] - 8s 6ms/step - loss: 1.3830 - accuracy: 0.4934 - val_loss: 1.2304 - val_accuracy: 0.5476
Epoch 3/25
1288/1288 [==============================] - 7s 5ms/step - loss: 1.1391 - accuracy: 0.5891 - val_loss: 1.2067 - val_accuracy: 0.5807
Epoch 4/25
1288/1288 [==============================] - 7s 5ms/step - loss: 0.9758 - accuracy: 0.6518 - val_loss: 0.9252 - val_accuracy: 0.6753
Epoch 5/25
1288/1288 [==============================] - 7s 5ms/step - loss: 0.8534 - accuracy: 0.7022 - val_loss: 0.8999 - val_accuracy: 0.6913
Epoch 6/25
1288/1288 [==============================] - 7s 5ms/step - loss: 0.7470 - accuracy: 0.7390 - val_loss: 0.8681 - val_accuracy: 0.7043
Epoch 7/25
1288/1288 [==============================] - 7s 5ms/step - loss: 0.6662 - accuracy: 0.7692 - val_loss: 0.8047 - val_accuracy:

In [ ]:
accuracy

0.7577333450317383

In [ ]:
gp_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.1114 - accuracy: 0.7520


[1.1113765239715576, 0.7519999742507935]

In [ ]:
gp_result.func_vals

In [ ]:
gp_result.x_iters

### Save the results

In [ ]:
df_temp = pd.concat([pd.DataFrame(gp_result.x_iters, columns = ["learning_rate", "num_conv_layers", "num_dense_layers",  
            "num_dense_nodes","activation", "batch_size"]), pd.Series(gp_result.func_vals*-100, name="accuracy")], axis=1)

In [ ]:
df_temp

In [ ]:
df_temp.to_csv("DF_cifar10_bayesian.csv")

## PT using Random search

In [ ]:
param_grid = {
              'dim_learning_rate' : list(np.logspace(np.log(0.005), np.log(0.2), base = np.exp(1), num = 1000)),
              'dim_num_conv_layers' : list(range(1, 3)),
              'dim_num_dense_layers' : list(range(1, 5)),
              'dim_num_dense_nodes' : list(range(1, 28)),
              'dim_activation' : ['relu', 'sigmoid'],
              'dim_batch_size' : list(range(1, 128))
}

In [ ]:
random.seed(50)

# Iterate through the specified number of evaluations
for i in range(5):
    
    # Randomly sample parameters for gbm
    params = [random.sample(value, 1)[0] for key, value in param_grid.items()]
    
    print(params)
    

[0.03275177220475209, 2, 3, 21, 'relu', 89]
[0.029863672437724486, 2, 1, 18, 'sigmoid', 29]
[0.06437322298735856, 1, 2, 12, 'relu', 126]
[0.01854686888813489, 2, 2, 7, 'relu', 125]
[0.13128416279545488, 2, 4, 21, 'relu', 118]


### Objective funtion for random search

In [ ]:
def fitness(learning_rate, num_conv_layers, num_dense_layers,  
            num_dense_nodes,activation, batch_size):

    model = create_model(learning_rate=learning_rate,
                         num_conv_layers = num_conv_layers,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         activation=activation
                        )

    early_S = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
    #named blackbox becuase it represents the structure
    blackbox = model.fit(x=X_train,
                        y=y_train,
                        epochs=25,
                        batch_size=batch_size,
                        validation_split=0.15,
                        callbacks = [early_S]
                        )
    #return the validation accuracy for the last epoch.
    accuracy = blackbox.history['val_accuracy'][-1]

    # Print the classification accuracy.
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    ops.reset_default_graph()
    
    return [learning_rate, num_conv_layers, num_dense_layers,  
            num_dense_nodes,activation, batch_size, accuracy]

### Run random search

In [ ]:
random.seed(50)

results = []
best_results = [0, 0, 0, 0, 0, 0, 0]

# Iterate through the specified number of evaluations
for i in range(50):

    print("Iteration: ", i+1)
    
    # Randomly sample parameters for gbm
    params = [random.sample(value, 1)[0] for key, value in param_grid.items()]
    
    result = fitness(params[0], params[1], params[2], params[3], params[4], params[5])
    results.append(result)
    if(best_results[6] < result[6]):
      best_results = result

### Results using random search

In [ ]:
results

In [ ]:
df_random = pd.DataFrame(results, 
                         columns = ['learning_rate', 'num_conv_layers', 'num_dense_layers',  
                              'num_dense_nodes','activation', 'batch_size', 'accuracy'])

In [ ]:
df_random.head()

,learning_rate,num_conv_layers,num_dense_layers,num_dense_nodes,activation,batch_size,accuracy
0,0.032752,2,3,21,relu,89,0.668400
1,0.029864,2,1,18,sigmoid,29,0.704400
2,0.064373,1,2,12,relu,126,0.586000
3,0.018547,2,2,7,relu,125,0.644667
4,0.131284,2,4,21,relu,118,0.670133


In [ ]:
df_random.to_csv("df_Cifar10_random.csv")

In [ ]:
gp_model = create_model(gp_result.x[0],gp_result.x[1],gp_result.x[2],gp_result.x[3],gp_result.x[4],gp_result.x[6])
gp_model.summary()

In [ ]:
model.summary()

### Retrain the best model architecture for random search

In [ ]:
gp_model.fit(X_train, y_train, batch_size=gp_result.x[5], epochs =20, validation_split=0.15)
gp_model.evaluate(X_train,y_train)

In [ ]:
gp_model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0830 - accuracy: 0.9836


[0.0829724669456482, 0.9836000204086304]

In [ ]:
best_results

[0.029863672437724486, 2, 1, 18, 'sigmoid', 29, 0.7044000029563904]

In [ ]:
model = create_model(best_results[0],best_results[1],best_results[2],best_results[3],best_results[4])

early_S = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#named blackbox becuase it represents the structure
blackbox = model.fit(x=X_train,
                    y=y_train,
                    epochs=25,
                    batch_size=best_results[5],
                    validation_split=0.15,
                    callbacks = [early_S]
                    )
#return the validation accuracy for the last epoch.
accuracy = blackbox.history['val_accuracy'][-1]

Epoch 1/25
1466/1466 [==============================] - 8s 5ms/step - loss: 1.8827 - accuracy: 0.3304 - val_loss: 1.9045 - val_accuracy: 0.3384
Epoch 2/25
1466/1466 [==============================] - 6s 4ms/step - loss: 1.4924 - accuracy: 0.4738 - val_loss: 1.3654 - val_accuracy: 0.5209
Epoch 3/25
1466/1466 [==============================] - 6s 4ms/step - loss: 1.2761 - accuracy: 0.5545 - val_loss: 1.3013 - val_accuracy: 0.5347
Epoch 4/25
1466/1466 [==============================] - 6s 4ms/step - loss: 1.1056 - accuracy: 0.6160 - val_loss: 1.0816 - val_accuracy: 0.6251
Epoch 5/25
1466/1466 [==============================] - 6s 4ms/step - loss: 0.9817 - accuracy: 0.6634 - val_loss: 1.0093 - val_accuracy: 0.6565
Epoch 6/25
1466/1466 [==============================] - 6s 4ms/step - loss: 0.8834 - accuracy: 0.6990 - val_loss: 0.9990 - val_accuracy: 0.6576
Epoch 7/25
1466/1466 [==============================] - 7s 4ms/step - loss: 0.8063 - accuracy: 0.7270 - val_loss: 0.9028 - val_accuracy:

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.8549 - accuracy: 0.7141


[0.8549267649650574, 0.7141000032424927]